#### for colab

In [1]:
# !pip install --upgrade torch
# !pip install transformers
# !pip install easydict
# !pip install colab-ssh --upgrade
# !pip install openpyxl

# from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
# launch_ssh_cloudflared(password='0000')

In [2]:
# from google.colab import drive

# # mount Google Drive
# drive.mount('/content/drive', force_remount=True)
# GDRIVE_HOME = '/content/drive/MyDrive'

## Experiments to do
21-05-17 12:29am
1. bert_attscore_bi_rnn
2. bert_attscore_rnn_add_cls

## Experiment options

In [3]:
## Experiment Option
from easydict import EasyDict
import torch

opt = EasyDict()
opt.dataset_series = 'SemEval-16' # SemEval-16, sentihood
opt.dataset_domain = 'restaurant' # restaurant / laptop
opt.subtask = 'sub1' # sub1: sentence, sub2: document(full review)
opt.task = 'category' # category, term
opt.num_classes = 3 # negative, positive, neutral, (+ conflict)
opt.max_length = 200
opt.model_name = 'bert_attscore_forcls_rnn'
# model_name: {bert_base, bert_attscore, bert_attscore_rnn, bert_attscore_bi_rnn, bert_attscore_rnn_add_asp,
#    bert_attscore_rnn_add_sep1, bert_attscore_rnn_add_sep_both, bert_attscore_forcls_rnn}
opt.pos = False
opt.lastid = False
opt.top_k = 3
opt.valset_ratio = 0.2
opt.batch_size = 16
opt.num_layers = 6
opt.num_epochs = 12
opt.runs = 5
opt.seed = 42
opt.log_step = 100
opt.patience = 5
opt.device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

print(opt.device)

cuda


## Load Dataset

In [4]:
import os, sys
# research_root = os.path.join(GDRIVE_HOME, 'research')
# sys.path.append(research_root)

if opt.dataset_series == 'SemEval-16':
    path = 'dataset/{}/semeval16_{}_{}.csv'.format(opt.dataset_series, opt.subtask, opt.dataset_domain)
    path_test = 'dataset/{}/semeval16_{}_{}_test.csv'.format(opt.dataset_series, opt.subtask, opt.dataset_domain)
elif opt.dataset_series == 'sentihood':
    path = 'dataset/{}/sentihood_train.csv'.format(opt.dataset_series)
    path_test = 'dataset/{}/sentihood_test.csv'.format(opt.dataset_series)

import pandas as pd

df_train = pd.read_csv(path)
df_test = pd.read_csv(path_test)

print('length of train set: {:,}'.format(len(df_train)))
print('length of test set: {:,}'.format(len(df_test)))

# df_train.head()

length of train set: 2,507
length of test set: 859


In [5]:
if 'attscore' in opt.model_name:
    from data_utils import clean_sentence, preprocess
    df_train = clean_sentence(df=df_train, clean_func=preprocess)
    df_test = clean_sentence(df=df_test, clean_func=preprocess)

In [6]:
from data_utils import Category_Classification_Dataset as Dataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
trainset = Dataset(df=df_train, tokenizer=tokenizer, opt=opt, pos_encoding=False)
testset = Dataset(df=df_test, tokenizer=tokenizer, opt=opt, pos_encoding=False)

# print(trainset.get_sample(423))
# print('-'*30)
# print(trainset[423])

sentihood: False
2,507 samples in this dataset
sentihood: False
859 samples in this dataset


In [7]:
from data_utils import custom_random_split as rs

train_set, val_set, test_set = rs(dataset=trainset, testset=testset,
                                  val_ratio=opt.valset_ratio, random_seed=opt.seed)

Ratio of datasets 2006 : 501 : 859


In [8]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_set, batch_size=opt.batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=opt.batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_set, batch_size=opt.batch_size, shuffle=False)

In [9]:
sample_batch = iter(train_loader).next()
sample_batch

{'input_ids': tensor([[[  101,  1996,  3295,  ...,     0,     0,     0]],
 
         [[  101,  2022,  2469,  ...,     0,     0,     0]],
 
         [[  101,  1996, 24857,  ...,     0,     0,     0]],
 
         ...,
 
         [[  101,  1996, 14163,  ...,     0,     0,     0]],
 
         [[  101,  2326,  2003,  ...,     0,     0,     0]],
 
         [[  101,  2204, 20861,  ...,     0,     0,     0]]]),
 'attention_masks': tensor([[[1, 1, 1,  ..., 0, 0, 0]],
 
         [[1, 1, 1,  ..., 0, 0, 0]],
 
         [[1, 1, 1,  ..., 0, 0, 0]],
 
         ...,
 
         [[1, 1, 1,  ..., 0, 0, 0]],
 
         [[1, 1, 1,  ..., 0, 0, 0]],
 
         [[1, 1, 1,  ..., 0, 0, 0]]]),
 'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0]],
 
         ...,
 
         [[0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0]]]),
 'labels': tensor([1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1

## Model

1. pair 단어들 (102번 사이)과 첫 문장 단어들 간의 attention score 합을 기준으로 top-k개 단어 선별
    - 그 단어들의 mean pool
    - 그 단어들을 rnn layer에?


In [10]:
opt.model_name

'bert_attscore_forcls_rnn'

In [11]:
from models.bert_intermediate import *
from models.bert_pos import *
from models.bert_attscores import *

if opt.model_name == 'bert_base':
    model = Bert_Base(opt.num_classes)
elif opt.model_name == 'bert_attscore':
    model = Bert_AttScore(opt=opt, embed_dim=768, fc_hid_dim=128, top_k=3, att_head='all', att_pooling='mean')
elif opt.model_name == 'bert_attscore_rnn':
    model = Bert_AttScore_RNN(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=False,
                              top_k=opt.top_k, att_head='all', att_pooling='gru')
elif opt.model_name == 'bert_attscore_bi_rnn':
    model = Bert_AttScore_RNN(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                              top_k=opt.top_k, att_head='all', att_pooling='gru')
elif opt.model_name == 'bert_attscore_rnn_add_sep1':
    model = Bert_AttScore_RNN_add(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', additional_token='sep1', att_pooling='gru')
elif opt.model_name == 'bert_attscore_rnn_add_sep2':
    model = Bert_AttScore_RNN_add(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', additional_token='sep2', att_pooling='gru')
elif opt.model_name == 'bert_attscore_rnn_add_sep_both':
    model = Bert_AttScore_RNN_add(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', additional_token='sep_both', att_pooling='gru')
elif opt.model_name == 'bert_attscore_rnn_add_asp':
    model = Bert_AttScore_RNN_add(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', additional_token='asp', att_pooling='gru')
elif opt.model_name == 'bert_attscore_rnn_add_cls':
    model = Bert_AttScore_RNN_add(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', additional_token='cls', att_pooling='gru')
elif opt.model_name == 'bert_attscore_forcls_rnn':
    model = Bert_AttScore_forCLS_RNN(opt=opt, embed_dim=768, rnn_hid_dim=256, fc_hid_dim=128, bidirectional=True,
                                 top_k=opt.top_k, att_head='all', att_pooling='gru')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
from models.parameters import get_parameters
total, params = get_parameters(model)

111,059,715 total parameters in this model
111,059,715 trainable parameters in this model


In [13]:
# add some tokens in vocab
# emb_layer = nn.Embedding(30524, 768, padding_idx=0)
# torch.nn.init.normal_(emb_layer.weight)
# model.bert.bert.embeddings.word_embeddings = emb_layer

## Train

In [14]:
import torch.nn as nn
import torch.optim as optim
from custom_trainer import *

optimizer = optim.AdamW(params, lr=2e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.8) # can't use for multiple runs
criterion = nn.CrossEntropyLoss()

result_dict, best_path = runs(trainer=trainer, train_loader=train_loader, val_loader=val_loader, test_loader=test_loader,
                             model=model, criterion=criterion, optimizer=optimizer, scheduler=False, opt=opt)

>>>>> RUN NUMBER: 01 <<<<<
   global step: 100 | train loss: 0.557, train_acc: 78.12%
Epoch: 01 | Val Loss: 0.031 | Val Acc: 82.44%
   global step: 200 | train loss: 0.272, train_acc: 90.71%
Epoch: 02 | Val Loss: 0.031 | Val Acc: 81.84%
   global step: 300 | train loss: 0.197, train_acc: 92.71%
>> saved: state_dict/bert_attscore_forcls_rnn_SemEval-16_restaurant_epoch_3_val_acc_79.44%
Epoch: 03 | Val Loss: 0.033 | Val Acc: 79.44%
   global step: 400 | train loss: 0.145, train_acc: 96.02%
   global step: 500 | train loss: 0.171, train_acc: 93.75%
>> saved: state_dict/bert_attscore_forcls_rnn_SemEval-16_restaurant_epoch_4_val_acc_81.44%
Epoch: 04 | Val Loss: 0.039 | Val Acc: 81.44%
   global step: 600 | train loss: 0.122, train_acc: 95.64%
Epoch: 05 | Val Loss: 0.041 | Val Acc: 80.04%
   global step: 700 | train loss: 0.106, train_acc: 95.27%
Epoch: 06 | Val Loss: 0.046 | Val Acc: 80.84%
   global step: 800 | train loss: 0.073, train_acc: 96.88%
Epoch: 07 | Val Loss: 0.048 | Val Acc: 80.4